In [1]:
import pandas as pd

finetuned_dirname = "40-epoch-bert-base-finetuned-twitter15-rnr"

data = pd.read_csv("../../data/processed/twitter15_dataset_with_tvt.csv", lineterminator="\n")
data = data[['tweet_text', 'tvt2', 'label']]
print(data.shape)
data.head()

(1490, 3)


,tweet_text,tvt2,label
0,🔥ca kkk grand wizard 🔥 endorses @hillaryclinto...,training,unverified
1,an open letter to trump voters from his top st...,training,unverified
2,america is a nation of second chances —@potus ...,validation,non-rumor
3,"brandon marshall visits and offers advice, sup...",testting,non-rumor
4,rip elly may clampett: so sad to learn #beverl...,validation,true


In [2]:
combined_data = data

In [3]:
import torch

class CustomTextDataset(torch.utils.data.dataset.Dataset):

    def __init__(self, texts, labels):
        self.labels = labels
        self.texts = texts
        self.attention_mask = None
        self.input_ids = None
        self.token_type_ids = None

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        sample = {
            "text": self.texts[idx],
            "label": self.labels[idx],
            "attention_mask": self.attention_mask[idx] if self.attention_mask else None,
            "input_ids": self.input_ids[idx] if self.input_ids else None,
            "token_type_ids": self.token_type_ids[idx] if self.token_type_ids else None
        }
        return sample
    
    def tokenize(self, tokenizer):
        self.attention_mask = []
        self.input_ids = []
        self.token_type_ids = []

        for text in self.texts:
            token = tokenizer(text, padding="max_length", truncation=True)
            
            self.attention_mask.append(token['attention_mask'])
            self.input_ids.append(token['input_ids'])
            self.token_type_ids.append(token['token_type_ids'])

c:\Users\msi_b\anaconda3\envs\pytorch-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
labels = []

labels_str = combined_data['label'].unique().tolist()
for i, d in combined_data.iterrows():
#     lab = labels_str.index(d['label'])
    if d['label'] in ['true', 'false', 'unverified']:
        lab = 0
    else:
        lab = 1
    labels.append(lab)
    
print(len(labels))
labels[:10]

1490


[0, 0, 1, 1, 0, 1, 0, 0, 0, 0]

In [5]:
train_dataset = CustomTextDataset(
    [d['tweet_text'] for i, d in combined_data.iterrows() if d['tvt2'] == 'training'],
    [labels[i] for i, d in combined_data.iterrows() if d['tvt2'] == 'training'])
test_dataset = CustomTextDataset(
    [d['tweet_text'] for i, d in combined_data.iterrows() if d['tvt2'] == 'validation'],
    [labels[i] for i, d in combined_data.iterrows() if d['tvt2'] == 'validation'])
train_dataset[0]

{'text': '🔥ca kkk grand wizard 🔥 endorses @hillaryclinton #neverhillary #trump2016 URL\r',
 'label': 0,
 'attention_mask': None,
 'input_ids': None,
 'token_type_ids': None}

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [7]:
# inputs = tokenizer(["you're stuck in a timewrap from 2004 though", "summa lumma dumma lumma"], padding="max_length", truncation=True)
# for k,v in inputs.items():
#     print(k)

In [8]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

train_dataset.tokenize(tokenizer)
test_dataset.tokenize(tokenizer)

In [9]:
print(len(train_dataset))
print(len(test_dataset))

1024
338


### Fine Tuning

In [10]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased",
                                                           output_hidden_states=False,
                                                           num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from transformers import TrainingArguments

epochs = 40
batch_size = 8
save_steps = (round((len(train_dataset)/batch_size) + 0.5)) * epochs
# save_steps = 1_000_000

training_args = TrainingArguments(
    output_dir=f"../../data/models/{finetuned_dirname}",
    num_train_epochs=epochs,
    save_steps=save_steps,
    logging_steps=300,
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy="epoch",
    logging_strategy="epoch"
)

print(f"Save Steps : {save_steps}")

Save Steps : 5120


In [12]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

C:\Users\msi_b\AppData\Local\Temp\ipykernel_13696\3043712826.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [13]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

In [14]:
import time

start = time.time()

trainer.train()

print(f"Execution Time : {round(time.time() - start)} seconds")

  2%|▎         | 128/5120 [00:42<26:08,  3.18it/s]

{'loss': 0.5559, 'learning_rate': 9.75e-06, 'epoch': 1.0}


                                                  
  2%|▎         | 128/5120 [00:46<26:08,  3.18it/s]

{'eval_loss': 0.5195600390434265, 'eval_accuracy': 0.7337278106508875, 'eval_runtime': 4.1378, 'eval_samples_per_second': 81.686, 'eval_steps_per_second': 10.392, 'epoch': 1.0}


  5%|▌         | 256/5120 [01:26<25:27,  3.18it/s]  

...

{'eval_loss': 1.3593791723251343, 'eval_accuracy': 0.849112426035503, 'eval_runtime': 4.1684, 'eval_samples_per_second': 81.086, 'eval_steps_per_second': 10.316, 'epoch': 39.0}


100%|██████████| 5120/5120 [29:43<00:00,  3.16it/s]

{'loss': 0.0, 'learning_rate': 0.0, 'epoch': 40.0}


                                                   
100%|██████████| 5120/5120 [29:47<00:00,  2.86it/s]

{'eval_loss': 1.3891386985778809, 'eval_accuracy': 0.849112426035503, 'eval_runtime': 4.1611, 'eval_samples_per_second': 81.228, 'eval_steps_per_second': 10.334, 'epoch': 40.0}
{'train_runtime': 1787.9838, 'train_samples_per_second': 22.908, 'train_steps_per_second': 2.864, 'train_loss': 0.03841363186561466, 'epoch': 40.0}
Execution Time : 1788 seconds


In [15]:
trainer.evaluate()

100%|██████████| 43/43 [00:04<00:00, 10.62it/s]


{'eval_loss': 1.3891386985778809,
 'eval_accuracy': 0.849112426035503,
 'eval_runtime': 4.1505,
 'eval_samples_per_second': 81.435,
 'eval_steps_per_second': 10.36,
 'epoch': 40.0}